# RAG Evaluation with DeepEval Framework

This notebook evaluates the Medical RAG chatbot using DeepEval framework.

## Evaluation Metrics:
1. **FaithfulnessMetric**: Checks if the answer is grounded in the retrieved context
2. **AnswerRelevancyMetric**: Checks if the answer is relevant to the question
3. **ContextualPrecisionMetric**: Measures precision of retrieved context
4. **ContextualRecallMetric**: Measures recall of retrieved context
5. **ContextualRelevancyMetric**: Checks if retrieved context is relevant to the question

In [22]:
import os
os.chdir('../')

In [23]:
# Install deepeval if not already installed
!pip install deepeval

In [24]:
# Import required libraries
from dotenv import load_dotenv
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings

# DeepEval imports
from deepeval.test_case import LLMTestCase
from deepeval import evaluate
from deepeval.metrics import (
    FaithfulnessMetric,
    AnswerRelevancyMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

load_dotenv()

False

In [25]:
# Set up environment variables
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [26]:
# Download embeddings function (as in trials.ipynb)
def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

# Initialize RAG components
print("Initializing embeddings...")
embedding = download_embeddings()

print("Connecting to Pinecone index...")
index_name = "medical-chatbot"
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

print("Creating retriever...")
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("Initializing chat model...")
chatModel = ChatOpenAI(model="gpt-4o")

print("Setting up prompt...")
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

print("Creating RAG chain...")
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

print("RAG system initialized successfully!")

Initializing embeddings...
Connecting to Pinecone index...
Creating retriever...
Initializing chat model...
Setting up prompt...
Creating RAG chain...
RAG system initialized successfully!


## Define Test Cases

Create test cases with medical questions that should be answerable from the medical book.

In [27]:
# Define test questions
test_questions = [
    "What is Acne?",
    "What is the treatment of Acne?",
    "What is Acromegaly and gigantism?",
    "What are the symptoms of diabetes?",
    "What is hypertension?",
    "How is pneumonia treated?",
    "What causes asthma?",
    "What are the side effects of common medications?"
]

answers = ["Acne is a common skin disorder caused by blocked hair follicles due to excess oil, dead skin cells, and bacteria. It commonly appears as pimples, blackheads, or cysts, especially on the face, chest, and back.",
"Treatment of acne includes topical agents like benzoyl peroxide or retinoids, oral antibiotics for infection, and hormonal therapy in severe cases. Proper skin hygiene also helps reduce outbreaks.",
"Acromegaly and gigantism are hormonal disorders caused by excessive growth hormone secretion, usually from a pituitary tumor. Gigantism occurs in children before bone growth stops, while acromegaly affects adults.",
"Common symptoms of diabetes include excessive thirst, frequent urination, unexplained weight loss, fatigue, and blurred vision. Poor wound healing and recurrent infections may also occur.",
"Hypertension is a condition characterized by persistently high blood pressure in the arteries. It increases the risk of heart disease, stroke, and kidney failure if left untreated.",
"Pneumonia is treated using antibiotics for bacterial infections, along with rest, fluids, and oxygen therapy if needed. Severe cases may require hospitalization and intravenous medication.",
"Asthma is caused by airway inflammation and hyper-responsiveness triggered by allergens, infections, exercise, or environmental pollutants. Genetic and environmental factors both play a role. ",
"Common medication side effects include nausea, dizziness, headache, allergic reactions, and gastrointestinal discomfort. The severity depends on the drug type, dosage, and patient sensitivity."


]

print(f"Defined {len(test_questions)} test questions")

Defined 8 test questions


In [28]:
# Function to get RAG response and retrieval context
def get_rag_response_and_context(question):
    """Get answer and retrieval context from RAG system."""
    # Get retrieval context
    retrieved_docs = retriever.invoke(question)
    retrieval_context = [doc.page_content for doc in retrieved_docs]
    
    # Get RAG response
    response = rag_chain.invoke({"input": question})
    answer = response.get("answer", "")
    
    return answer, retrieval_context

In [29]:
# Generate test cases
print("Generating test cases...")
test_cases = []

for question in test_questions:
    print(f"Processing: {question}")
    actual_output, retrieval_context = get_rag_response_and_context(question)
    
    test_case = LLMTestCase(
        input=question,
        actual_output=actual_output,
        retrieval_context=retrieval_context
    )
    test_cases.append(test_case)
    print(f"✓ Generated test case for: {question[:50]}...")

print(f"\nGenerated {len(test_cases)} test cases")

Generating test cases...
Processing: What is Acne?
✓ Generated test case for: What is Acne?...
Processing: What is the treatment of Acne?
✓ Generated test case for: What is the treatment of Acne?...
Processing: What is Acromegaly and gigantism?
✓ Generated test case for: What is Acromegaly and gigantism?...
Processing: What are the symptoms of diabetes?
✓ Generated test case for: What are the symptoms of diabetes?...
Processing: What is hypertension?
✓ Generated test case for: What is hypertension?...
Processing: How is pneumonia treated?
✓ Generated test case for: How is pneumonia treated?...
Processing: What causes asthma?
✓ Generated test case for: What causes asthma?...
Processing: What are the side effects of common medications?
✓ Generated test case for: What are the side effects of common medications?...

Generated 8 test cases


In [30]:
# Update test cases with expected_output (ground truth answers)
print("Updating test cases with expected_output...")
for i, test_case in enumerate(test_cases):
    if i < len(answers):
        test_case.expected_output = answers[i]
        print(f"✓ Added expected_output to test case {i+1}: {test_questions[i][:50]}...")
    else:
        print(f"⚠ No expected_output available for test case {i+1}")

print(f"\nUpdated {len([tc for tc in test_cases if tc.expected_output])} test cases with expected_output")

Updating test cases with expected_output...
✓ Added expected_output to test case 1: What is Acne?...
✓ Added expected_output to test case 2: What is the treatment of Acne?...
✓ Added expected_output to test case 3: What is Acromegaly and gigantism?...
✓ Added expected_output to test case 4: What are the symptoms of diabetes?...
✓ Added expected_output to test case 5: What is hypertension?...
✓ Added expected_output to test case 6: How is pneumonia treated?...
✓ Added expected_output to test case 7: What causes asthma?...
✓ Added expected_output to test case 8: What are the side effects of common medications?...

Updated 8 test cases with expected_output


In [31]:
# Display sample test case
if test_cases:
    print("Sample Test Case:")
    print(f"Input: {test_cases[0].input}")
    print(f"Output: {test_cases[0].actual_output[:200]}...")
    print(f"Retrieval Context (first chunk): {test_cases[0].retrieval_context[0][:200] if test_cases[0].retrieval_context else 'None'}...")

Sample Test Case:
Input: What is Acne?
Output: Acne is a skin disorder where the sebaceous glands become inflamed, leading to the development of pimples and other lesions. It is commonly referred to as acne vulgaris....
Retrieval Context (first chunk): GALE ENCYCLOPEDIA OF MEDICINE 226
Acne
GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26...


## Prepare Test Cases for Evaluation

In [32]:
# Test cases are ready for evaluation
print(f"Created {len(test_cases)} test cases ready for evaluation")

Created 8 test cases ready for evaluation


## Define Evaluation Metrics

In [33]:
# Initialize metrics
faithfulness_metric = FaithfulnessMetric(threshold=0.7)
answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.7)
contextual_precision_metric = ContextualPrecisionMetric(threshold=0.7)
contextual_recall_metric = ContextualRecallMetric(threshold=0.7)
contextual_relevancy_metric = ContextualRelevancyMetric(threshold=0.7)

metrics = [
    faithfulness_metric,
    answer_relevancy_metric,
    contextual_precision_metric,
    contextual_recall_metric,
    contextual_relevancy_metric
]

print("Initialized evaluation metrics:")
for metric in metrics:
    print(f"  - {metric.__class__.__name__}")

Initialized evaluation metrics:
  - FaithfulnessMetric
  - AnswerRelevancyMetric
  - ContextualPrecisionMetric
  - ContextualRecallMetric
  - ContextualRelevancyMetric


## Run Evaluation

In [34]:
# Run evaluation
print("Starting evaluation...")
print("This may take a few minutes...\n")

evaluate(
    test_cases=test_cases,
    metrics=metrics,
    skip_on_missing_params=True  # Safety parameter
)

print("\n✓ Evaluation completed!")

Starting evaluation...
This may take a few minutes...



TypeError: evaluate() got an unexpected keyword argument 'skip_on_missing_params'

## Individual Metric Evaluation

Evaluate each metric individually to get detailed results.

In [ ]:
# Evaluate Faithfulness Metric
print("Evaluating Faithfulness Metric...")
faithfulness_results = []

for i, test_case in enumerate(test_cases):
    try:
        faithfulness_metric.measure(test_case)
        faithfulness_results.append({
            'question': test_case.input,
            'score': faithfulness_metric.score,
            'reason': faithfulness_metric.reason,
            'success': faithfulness_metric.success
        })
        print(f"  Test {i+1}: {test_case.input[:50]}... - Score: {faithfulness_metric.score:.3f} - {'✓' if faithfulness_metric.success else '✗'}")
    except Exception as e:
        print(f"  Test {i+1}: Error - {str(e)}")
        faithfulness_results.append({
            'question': test_case.input,
            'score': None,
            'reason': str(e),
            'success': False
        })

print(f"\nFaithfulness Average Score: {sum([r['score'] for r in faithfulness_results if r['score'] is not None]) / len([r for r in faithfulness_results if r['score'] is not None]):.3f}")
print(f"Success Rate: {sum([1 for r in faithfulness_results if r['success']]) / len(faithfulness_results) * 100:.1f}%")

In [ ]:
# Evaluate Answer Relevancy Metric
print("Evaluating Answer Relevancy Metric...")
answer_relevancy_results = []

for i, test_case in enumerate(test_cases):
    try:
        answer_relevancy_metric.measure(test_case)
        answer_relevancy_results.append({
            'question': test_case.input,
            'score': answer_relevancy_metric.score,
            'reason': answer_relevancy_metric.reason,
            'success': answer_relevancy_metric.success
        })
        print(f"  Test {i+1}: {test_case.input[:50]}... - Score: {answer_relevancy_metric.score:.3f} - {'✓' if answer_relevancy_metric.success else '✗'}")
    except Exception as e:
        print(f"  Test {i+1}: Error - {str(e)}")
        answer_relevancy_results.append({
            'question': test_case.input,
            'score': None,
            'reason': str(e),
            'success': False
        })

print(f"\nAnswer Relevancy Average Score: {sum([r['score'] for r in answer_relevancy_results if r['score'] is not None]) / len([r for r in answer_relevancy_results if r['score'] is not None]):.3f}")
print(f"Success Rate: {sum([1 for r in answer_relevancy_results if r['success']]) / len(answer_relevancy_results) * 100:.1f}%")

In [ ]:
# Evaluate Contextual Precision Metric
print("Evaluating Contextual Precision Metric...")
contextual_precision_results = []

for i, test_case in enumerate(test_cases):
    try:
        contextual_precision_metric.measure(test_case)
        contextual_precision_results.append({
            'question': test_case.input,
            'score': contextual_precision_metric.score,
            'reason': contextual_precision_metric.reason,
            'success': contextual_precision_metric.success
        })
        print(f"  Test {i+1}: {test_case.input[:50]}... - Score: {contextual_precision_metric.score:.3f} - {'✓' if contextual_precision_metric.success else '✗'}")
    except Exception as e:
        print(f"  Test {i+1}: Error - {str(e)}")
        contextual_precision_results.append({
            'question': test_case.input,
            'score': None,
            'reason': str(e),
            'success': False
        })

print(f"\nContextual Precision Average Score: {sum([r['score'] for r in contextual_precision_results if r['score'] is not None]) / len([r for r in contextual_precision_results if r['score'] is not None]):.3f}")
print(f"Success Rate: {sum([1 for r in contextual_precision_results if r['success']]) / len(contextual_precision_results) * 100:.1f}%")

In [ ]:
# Evaluate Contextual Recall Metric
print("Evaluating Contextual Recall Metric...")
contextual_recall_results = []

for i, test_case in enumerate(test_cases):
    try:
        contextual_recall_metric.measure(test_case)
        contextual_recall_results.append({
            'question': test_case.input,
            'score': contextual_recall_metric.score,
            'reason': contextual_recall_metric.reason,
            'success': contextual_recall_metric.success
        })
        print(f"  Test {i+1}: {test_case.input[:50]}... - Score: {contextual_recall_metric.score:.3f} - {'✓' if contextual_recall_metric.success else '✗'}")
    except Exception as e:
        print(f"  Test {i+1}: Error - {str(e)}")
        contextual_recall_results.append({
            'question': test_case.input,
            'score': None,
            'reason': str(e),
            'success': False
        })

print(f"\nContextual Recall Average Score: {sum([r['score'] for r in contextual_recall_results if r['score'] is not None]) / len([r for r in contextual_recall_results if r['score'] is not None]):.3f}")
print(f"Success Rate: {sum([1 for r in contextual_recall_results if r['success']]) / len(contextual_recall_results) * 100:.1f}%")

In [ ]:
# Evaluate Contextual Relevancy Metric
print("Evaluating Contextual Relevancy Metric...")
contextual_relevancy_results = []

for i, test_case in enumerate(test_cases):
    try:
        contextual_relevancy_metric.measure(test_case)
        contextual_relevancy_results.append({
            'question': test_case.input,
            'score': contextual_relevancy_metric.score,
            'reason': contextual_relevancy_metric.reason,
            'success': contextual_relevancy_metric.success
        })
        print(f"  Test {i+1}: {test_case.input[:50]}... - Score: {contextual_relevancy_metric.score:.3f} - {'✓' if contextual_relevancy_metric.success else '✗'}")
    except Exception as e:
        print(f"  Test {i+1}: Error - {str(e)}")
        contextual_relevancy_results.append({
            'question': test_case.input,
            'score': None,
            'reason': str(e),
            'success': False
        })

print(f"\nContextual Relevancy Average Score: {sum([r['score'] for r in contextual_relevancy_results if r['score'] is not None]) / len([r for r in contextual_relevancy_results if r['score'] is not None]):.3f}")
print(f"Success Rate: {sum([1 for r in contextual_relevancy_results if r['success']]) / len(contextual_relevancy_results) * 100:.1f}%")

## Summary Report

In [ ]:
# Create summary report
import pandas as pd

summary_data = {
    'Metric': [
        'Faithfulness',
        'Answer Relevancy',
        'Contextual Precision',
        'Contextual Recall',
        'Contextual Relevancy'
    ],
    'Average Score': [
        sum([r['score'] for r in faithfulness_results if r['score'] is not None]) / len([r for r in faithfulness_results if r['score'] is not None]) if any(r['score'] is not None for r in faithfulness_results) else 0,
        sum([r['score'] for r in answer_relevancy_results if r['score'] is not None]) / len([r for r in answer_relevancy_results if r['score'] is not None]) if any(r['score'] is not None for r in answer_relevancy_results) else 0,
        sum([r['score'] for r in contextual_precision_results if r['score'] is not None]) / len([r for r in contextual_precision_results if r['score'] is not None]) if any(r['score'] is not None for r in contextual_precision_results) else 0,
        sum([r['score'] for r in contextual_recall_results if r['score'] is not None]) / len([r for r in contextual_recall_results if r['score'] is not None]) if any(r['score'] is not None for r in contextual_recall_results) else 0,
        sum([r['score'] for r in contextual_relevancy_results if r['score'] is not None]) / len([r for r in contextual_relevancy_results if r['score'] is not None]) if any(r['score'] is not None for r in contextual_relevancy_results) else 0
    ],
    'Success Rate (%)': [
        sum([1 for r in faithfulness_results if r['success']]) / len(faithfulness_results) * 100,
        sum([1 for r in answer_relevancy_results if r['success']]) / len(answer_relevancy_results) * 100,
        sum([1 for r in contextual_precision_results if r['success']]) / len(contextual_precision_results) * 100,
        sum([1 for r in contextual_recall_results if r['success']]) / len(contextual_recall_results) * 100,
        sum([1 for r in contextual_relevancy_results if r['success']]) / len(contextual_relevancy_results) * 100
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n" + "="*60)
print("EVALUATION SUMMARY REPORT")
print("="*60)
print(summary_df.to_string(index=False))
print("="*60)

In [ ]:
# Display detailed results for each test case
print("\n" + "="*60)
print("DETAILED RESULTS BY TEST CASE")
print("="*60)

for i, test_case in enumerate(test_cases):
    print(f"\nTest Case {i+1}: {test_case.input}")
    print("-" * 60)
    
    if i < len(faithfulness_results):
        print(f"Faithfulness: {faithfulness_results[i]['score']:.3f if faithfulness_results[i]['score'] else 'N/A'} - {'✓' if faithfulness_results[i]['success'] else '✗'}")
    
    if i < len(answer_relevancy_results):
        print(f"Answer Relevancy: {answer_relevancy_results[i]['score']:.3f if answer_relevancy_results[i]['score'] else 'N/A'} - {'✓' if answer_relevancy_results[i]['success'] else '✗'}")
    
    if i < len(contextual_precision_results):
        print(f"Contextual Precision: {contextual_precision_results[i]['score']:.3f if contextual_precision_results[i]['score'] else 'N/A'} - {'✓' if contextual_precision_results[i]['success'] else '✗'}")
    
    if i < len(contextual_recall_results):
        print(f"Contextual Recall: {contextual_recall_results[i]['score']:.3f if contextual_recall_results[i]['score'] else 'N/A'} - {'✓' if contextual_recall_results[i]['success'] else '✗'}")
    
    if i < len(contextual_relevancy_results):
        print(f"Contextual Relevancy: {contextual_relevancy_results[i]['score']:.3f if contextual_relevancy_results[i]['score'] else 'N/A'} - {'✓' if contextual_relevancy_results[i]['success'] else '✗'}")
    
    print(f"\nAnswer: {test_case.actual_output[:200]}...")

## Notes

- **Faithfulness**: Measures if the answer is grounded in the retrieved context (no hallucinations)
- **Answer Relevancy**: Measures if the answer is relevant to the question
- **Contextual Precision**: Measures precision of retrieved context (how many retrieved chunks are relevant)
- **Contextual Recall**: Measures recall of retrieved context (how much relevant information was retrieved)
- **Contextual Relevancy**: Measures if the retrieved context is relevant to the question

All metrics use a threshold of 0.7. A test case passes if the score is >= 0.7.